In [1]:
import scanpy as sc

In [15]:
# Read scRNA-seq data
adata = sc.read_h5ad("SEAAD_MTG_RNAseq_final-nuclei.2024-02-13.h5ad", backed='r')

In [3]:
print(adata.obs.columns)

Index(['sample_id', 'Neurotypical reference', 'Donor ID', 'Organism',
       'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)',
       'Race (choice=Black/ African American)',
       ...
       'Used in analysis', 'Class confidence', 'Class', 'Subclass confidence',
       'Subclass', 'Supertype confidence', 'Supertype (non-expanded)',
       'Supertype', 'Continuous Pseudo-progression Score',
       'Severely Affected Donor'],
      dtype='object', length=135)


In [4]:
print(adata.obs["Brain Region"].unique())

['Human MTG', 'Human MTG_L5', 'Human MTG All Layers']
Categories (3, object): ['Human MTG', 'Human MTG All Layers', 'Human MTG_L5']


In [5]:
print(f"Total cells: {adata.n_obs}")
print(f"Total genes: {adata.n_vars}")

Total cells: 1378211
Total genes: 36601


In [6]:
mask = adata.obs["Brain Region"] == "Human MTG_L5"
selected_cells = adata.obs_names[mask]

print("Number of cells:", len(selected_cells))

Number of cells: 34922


In [7]:
adata_MTG_L5 = sc.read_h5ad("SEAAD_MTG_RNAseq_final-nuclei.2024-02-13.h5ad", backed='r')[selected_cells, :].to_memory()

In [8]:
adata_MTG_L5.obs.columns

Index(['sample_id', 'Neurotypical reference', 'Donor ID', 'Organism',
       'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)',
       'Race (choice=Black/ African American)',
       ...
       'Used in analysis', 'Class confidence', 'Class', 'Subclass confidence',
       'Subclass', 'Supertype confidence', 'Supertype (non-expanded)',
       'Supertype', 'Continuous Pseudo-progression Score',
       'Severely Affected Donor'],
      dtype='object', length=135)

In [9]:
adata_MTG_L5.obs["Subclass"].value_counts()

Subclass
L5 IT              8807
L4 IT              4613
Sst                4395
Pvalb              4336
L6 IT Car3         3068
L6 IT              1787
L5/6 NP            1620
Vip                1403
L6b                1120
L2/3 IT            1049
L6 CT               840
Lamp5 Lhx6          830
Lamp5               516
L5 ET               216
Chandelier          164
Sst Chodl            41
Oligodendrocyte      36
Pax6                 30
Endothelial          16
Astrocyte            14
Sncg                  7
Microglia-PVM         7
OPC                   6
VLMC                  1
Name: count, dtype: int64

In [10]:
adata_MTG_L5.obs["Class"].value_counts()

Class
Neuronal: Glutamatergic        23120
Neuronal: GABAergic            11722
Non-neuronal and Non-neural       80
Name: count, dtype: int64

In [11]:
# Normalize and log-transform
sc.pp.normalize_total(adata_MTG_L5, target_sum=1e4)
sc.pp.log1p(adata_MTG_L5)

# Select top 2000 highly variable genes (optional but recommended)
sc.pp.highly_variable_genes(adata_MTG_L5, n_top_genes=2000, subset=True)

In [12]:
sc.tl.rank_genes_groups(
    adata_MTG_L5,
    groupby="Subclass",
    groups=["L5 IT"],     # cell type of interest
    reference="rest",     # compare vs all other subclasses
    method="wilcoxon"
)

In [13]:
# identify genes that are differentially expressed in L5 IT vs other cell types
deg = sc.get.rank_genes_groups_df(adata_MTG_L5, group="L5 IT")
deg.head(20)

,names,scores,logfoldchanges,pvals,pvals_adj
0,IL1RAPL2,108.958481,2.702137,0.0,0.0
1,CASC15,108.690010,2.023123,0.0,0.0
2,CPNE4,105.508072,1.810703,0.0,0.0
3,LRRK1,100.498947,2.297281,0.0,0.0
4,IPCEF1,100.253082,2.030481,0.0,0.0
5,LDLRAD3,99.464058,2.827661,0.0,0.0
6,HS3ST2,99.157211,2.084499,0.0,0.0
7,TLL1,97.960358,2.742297,0.0,0.0
8,RXFP1,96.707504,1.977844,0.0,0.0
9,RORB,95.499580,2.749091,0.0,0.0


In [16]:
# 1. Open in backed mode to avoid loading everything, read scATAC-seq data
adata = sc.read_h5ad("SEAAD_MTG_ATACseq_final-nuclei.2024-12-06.h5ad", backed="r")

In [17]:
print(adata.obs.columns)

Index(['sample_id', 'Neurotypical reference', 'Donor ID', 'Organism',
       'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)',
       'Race (choice=Black/ African American)',
       ...
       'Class', 'Subclass confidence', 'Subclass', 'Supertype confidence',
       'Supertype (non-expanded)', 'Supertype', 'RNA Quality Control Score',
       'Quality Control Clusters', 'Continuous Pseudo-progression Score',
       'Severely Affected Donor'],
      dtype='object', length=129)


In [18]:
print(f"Total cells: {adata.n_obs}")
print(f"Total peaks: {adata.n_vars}")

Total cells: 516389
Total peaks: 218882


In [19]:
adata.obs["Subclass"].value_counts()

Subclass
L2/3 IT            143333
L4 IT               65159
L5 IT               48096
Vip                 36557
Pvalb               36282
Oligodendrocyte     35233
Sst                 26008
Astrocyte           19636
L6 IT               18617
Lamp5               16411
L6 IT Car3          10021
Microglia-PVM        9663
OPC                  8563
Sncg                 8413
Lamp5 Lhx6           7660
L5/6 NP              7187
L6 CT                5797
L6b                  5001
Pax6                 2884
Chandelier           2827
VLMC                 1261
L5 ET                 993
Sst Chodl             488
Endothelial           299
Name: count, dtype: int64

In [21]:
import snapatac2 as snap
# run pip install snapatac2 if not install yet

In [9]:
print(adata.var.head()) 

Empty DataFrame
Columns: []
Index: [chr4:164130572-164131513, chr4:16412872-16415576, chr4:164116366-164116739, chr4:164083047-164083256, chr4:164056041-164057013]


In [24]:
import numpy as np

In [22]:
target = "Astrocyte"
group_col = "Subclass"

mask_target = (adata.obs[group_col] == target)

In [25]:
# building a background set of random cells from all other cell types except the target cell type
barcodes = np.array(adata.obs_names)
background = []

for ct in adata.obs[group_col].unique():
    if ct == target:
        continue
    mask = (adata.obs[group_col] == ct)
    n = mask.sum()
    if n > 0:
        k = min(50, n)  # up to 50 per cell type
        background.append(np.random.choice(barcodes[mask], size=k, replace=False))

background = np.concatenate(background)
mask_background = np.isin(adata.obs_names, background)

In [ ]:
# identify open chromatin regions that are selectively enriched in astrocyte, this will takes a while
da = snap.tl.diff_test(
    adata,
    cell_group1 = mask_target,
    cell_group2 = mask_background,
    direction = "positive"     # enriched in cell_group1
)

2025-11-20 23:09:17 - INFO - Input contains 218882 features, now perform filtering with 'min_log_fc = 0.25' and 'min_pct = 0.05' ...
2025-11-20 23:09:52 - INFO - Testing 30584 features ...
 40%|████      | 12263/30584 [12:59<18:44, 16.29it/s]

In [9]:
# to get results returned by SnapATAC2
import polars as pl

In [10]:
# keep only statistically significant peaks
da_sig = da.filter(pl.col("adjusted p-value") < 0.01)

print(da_sig.head())

shape: (5, 4)
┌──────────────────────────┬───────────────────┬─────────┬──────────────────┐
│ feature name             ┆ log2(fold_change) ┆ p-value ┆ adjusted p-value │
│ ---                      ┆ ---               ┆ ---     ┆ ---              │
│ str                      ┆ f32               ┆ f64     ┆ f64              │
╞══════════════════════════╪═══════════════════╪═════════╪══════════════════╡
│ chr4:1789181-1815273     ┆ 2.38265           ┆ 0.0     ┆ 0.0              │
│ chr4:1775512-1781064     ┆ 3.34286           ┆ 0.0     ┆ 0.0              │
│ chr4:2785634-2802095     ┆ 2.295867          ┆ 0.0     ┆ 0.0              │
│ chr5:38465985-38470039   ┆ 3.489634          ┆ 0.0     ┆ 0.0              │
│ chr5:139805058-139809732 ┆ 3.697418          ┆ 0.0     ┆ 0.0              │
└──────────────────────────┴───────────────────┴─────────┴──────────────────┘
